### Description
This notebook is an exercise to study the speed at which the Ascendant (`Lagna`) moves per sec, on average over the entire year. Please remember to install the following packages in your virtual environment before running this notebook.

```bash
pip install plotly
pip install --upgrade nbformat
```

In [15]:
from flatlib import const
import plotly.graph_objects as go
from datetime import datetime, timedelta
from vedicastro.VedicAstro import VedicHoroscopeData

In [16]:
def calculate_daily_asc_speed(year, lat, lon, utc, ayan, house_system):
    """Calculates the ascendant speed for each day of the year."""
    asc_speeds = []
    dates = []
    
    # Loop through each day of the year
    start_date = datetime(year, 1, 1)
    for day in range(365):
        current_date = start_date + timedelta(days=day)
        dates.append(current_date.strftime('%Y-%m-%d'))
        
        # Define two moments in time, at the start and end of the day
        datetime_start = current_date.replace(hour=0, minute=0, second=1)
        datetime_end = current_date.replace(hour=0, minute=0, second=11)
        # datetime_end = current_date.replace(hour=23, minute=59, second=59)
        difference = datetime_end - datetime_start
        diff_seconds = difference.total_seconds()

        
        # Generate horoscope data for both moments
        horoscope_start = VedicHoroscopeData(datetime_start.year, datetime_start.month, datetime_start.day, datetime_start.hour, datetime_start.minute, datetime_start.second, utc, lat, lon, ayan, house_system)
        final_chart_start = horoscope_start.generate_chart()
        asc_start = final_chart_start.get(const.ASC)
        asc_start_lon_deg = asc_start.lon
        
        horoscope_end = VedicHoroscopeData(datetime_end.year, datetime_end.month, datetime_end.day, datetime_end.hour, datetime_end.minute, datetime_end.second, utc, lat, lon, ayan, house_system)
        final_chart_end = horoscope_end.generate_chart()
        asc_end = final_chart_end.get(const.ASC)
        asc_end_lon_deg = asc_end.lon
        

        # Calculate the change in Ascendant position
        delta_degrees = (asc_end_lon_deg - asc_start_lon_deg) % 360
        
        
        asc_speeds.append(delta_degrees/ diff_seconds)  # Speed in degrees per second over the entire day
    
    return dates, asc_speeds

In [17]:
# Example usage parameters
year = 2024
lat = 11.020085773931049 # Example latitude
lon = 76.98319647719487  # Example longitude
utc = "+05:30"
ayan = "Krishnamurti" 
house_system = "Placidus"

dates, asc_speeds = calculate_daily_asc_speed(year, lat, lon, utc, ayan, house_system)
print(f"Asc Speed on {dates[0]}: {asc_speeds[0]:.5f} ° per second")



Asc Speed on 2024-01-01: 0.00420 ° per second


In [18]:
# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=asc_speeds, mode='lines',name = "",
                         hovertemplate="<b>Date</b>: %{x}<br>" + "<b>AscSpeed</b>: %{y:.5f}° per second"))

fig.update_layout(title=f'Ascendant Movement Per Second for Each Day of {year}',
                    xaxis_title='Date',
                    yaxis_title='Movement (degrees/sec)',
                    template='plotly_dark')
fig.show()